In [1]:
# Data stuff
import numpy as np
import pandas as pd

# Visual stuff
from IPython.display import display
# Configs 
pd.options.display.max_columns = None
pd.options.display.max_rows = None
import matplotlib.pyplot as plt

# Random state seed
rand_state=42

# ML stuff
import sklearn
from sklearn import metrics
from sklearn.model_selection import RandomizedSearchCV,StratifiedKFold
import lightgbm as lgb
import xgboost as xgb
from xgboost import XGBClassifier
import shap

In [2]:
dataset = pd.read_csv('data_gen/dataset_missing_entire.csv')
# Ensure correct types
dataset = dataset.astype(dtype={
    'age':float,
    'gender':int,
    'expired':int,
    'P-glucose':float,
    'blood_pressure_systoliskt':float,
    'blood_pressure_diastoliskt':float,
    'BMI':float
    
})
# Order ints(categorical variables, first) and floats(number variables, last)
dataset = dataset[[
    'gender',
    'I109',
    'E119',
    'E669',
    'I259',
    'I252',
    'I209',
    'E660',
    'E118',
    'I639',
    'E113',
    'expired',
    'age',
    'P-glucose',
    'blood_pressure_systoliskt',
    'blood_pressure_diastoliskt',
    'BMI',
]]

In [3]:
# Generate data set withouth categories(all numbers)
Y_no_cat = dataset.expired.values
X_no_cat = dataset.drop(columns=['expired']).values

# dataset as is, but target variable dropped(hospital expire flag)
dataset_no_target = dataset.drop(columns=['expired'])

# Feture names and categorical feature names
feature_names = dataset_no_target.select_dtypes(include='int').columns.values.tolist() + dataset_no_target.select_dtypes(exclude='int').columns.values.tolist() 
cat_feature_names = dataset_no_target.select_dtypes(include='int').columns.values.tolist() 

# Generate data set with categories(int type required)
dataframe_int_list = dataset_no_target.select_dtypes(include='int').values.tolist()
dataframe_no_int_list = dataset_no_target.select_dtypes(exclude='int').values.tolist()
Y = dataset.expired.values.tolist()
X = []
for i,v in enumerate(dataframe_int_list):
    X = X + [v+dataframe_no_int_list[i]]

# Generate categorical feature indicies
cat_features_indices=list(range(0,len(dataframe_int_list[0])))

In [4]:
# Function that w
def strat_cv_it(classifier, params, uses_cat, param_comb ):
    folds = 3
    skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = rand_state)
    
    # Perform fit and scoring
    if uses_cat:
        random_search = RandomizedSearchCV( classifier, return_train_score=True, param_distributions=params ,scoring='roc_auc', n_iter=param_comb, n_jobs=4, cv=skf.split(X, Y),random_state = rand_state, refit=True, verbose=3, error_score=0.0 )
        random_search.fit(X, Y)     
    else:
        random_search = RandomizedSearchCV( classifier, return_train_score=True, param_distributions=params ,scoring='roc_auc', n_iter=param_comb, n_jobs=4, cv=skf.split(X_no_cat, Y_no_cat),random_state = rand_state, refit=True, verbose=3,error_score=0.0)
        random_search.fit(X_no_cat, Y_no_cat)
    
    # Display results and return best model
    display(random_search.best_score_)
    display(random_search.best_params_)
    display(pd.DataFrame(random_search.cv_results_))
    return random_search.best_estimator_

# Scale of negative class to the positive class(#survived/#died)
scale_pos_weight_min = int( (dataset[dataset.expired==0].shape[0] / dataset[dataset.expired==1].shape[0]) )

# XGboost

In [5]:
xgb_classifier = xgb.XGBClassifier(objective = "binary:logistic",random_state=rand_state)
xgb_params = {
        'learning_rate': (0.01, 0.05,0.1),
        'min_child_weight': [3, 5, 10],
        'gamma': [0.5, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': list(range(5,30)),
        'scale_pos_weight':  list(range(scale_pos_weight_min,3*scale_pos_weight_min))
}
model = strat_cv_it(xgb_classifier ,xgb_params,False,50)
explainer = shap.TreeExplainer(model)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:  4.1min
[Parallel(n_jobs=4)]: Done 120 tasks      | elapsed:  8.6min
[Parallel(n_jobs=4)]: Done 150 out of 150 | elapsed:  9.2min finished


0.8791764021250468

{'subsample': 0.6,
 'scale_pos_weight': 25,
 'min_child_weight': 10,
 'max_depth': 28,
 'learning_rate': 0.01,
 'gamma': 2,
 'colsample_bytree': 0.8}

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_subsample,param_scale_pos_weight,param_min_child_weight,param_max_depth,param_learning_rate,param_gamma,param_colsample_bytree,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,mean_train_score,std_train_score
0,28.033031,1.144128,0.054735,0.030769,1,22,5,7,0.01,1.5,1,"{'subsample': 1.0, 'scale_pos_weight': 22, 'mi...",0.864641,0.876224,0.855049,0.865305,0.008657,19,0.943811,0.944859,0.943911,0.944194,0.000472
1,59.648254,2.402599,0.064932,0.009438,1,25,10,20,0.1,2,1,"{'subsample': 1.0, 'scale_pos_weight': 25, 'mi...",0.846930,0.846982,0.837812,0.843908,0.004311,44,0.995625,0.995022,0.994321,0.994989,0.000533
2,47.145547,2.637916,0.053737,0.012904,0.8,27,10,12,0.1,1.5,1,"{'subsample': 0.8, 'scale_pos_weight': 27, 'mi...",0.855893,0.852948,0.848564,0.852468,0.003011,37,0.991993,0.992284,0.991402,0.991893,0.000367
3,18.150708,2.414671,0.056225,0.002264,1,14,3,6,0.1,5,0.8,"{'subsample': 1.0, 'scale_pos_weight': 14, 'mi...",0.867641,0.867290,0.850236,0.861722,0.008123,27,0.983457,0.980687,0.980221,0.981455,0.001428
4,58.500879,6.286872,0.071195,0.013925,1,29,10,20,0.1,0.5,1,"{'subsample': 1.0, 'scale_pos_weight': 29, 'mi...",0.848845,0.848037,0.837923,0.844935,0.004969,43,0.996530,0.996177,0.995145,0.995951,0.000588
5,52.029134,9.352539,0.046757,0.012887,0.6,26,5,17,0.01,0.5,1,"{'subsample': 0.6, 'scale_pos_weight': 26, 'mi...",0.873121,0.873013,0.868745,0.871626,0.002038,8,0.958711,0.961269,0.960366,0.960116,0.001059
6,29.303578,5.812649,0.056104,0.006527,0.8,25,10,9,0.01,0.5,0.8,"{'subsample': 0.8, 'scale_pos_weight': 25, 'mi...",0.884752,0.874998,0.870649,0.876800,0.005897,3,0.953602,0.955791,0.956711,0.955368,0.001304
7,24.539514,0.946734,0.044559,0.012247,0.6,29,10,17,0.01,2,1,"{'subsample': 0.6, 'scale_pos_weight': 29, 'mi...",0.882184,0.874368,0.874411,0.876987,0.003674,2,0.943790,0.948146,0.947504,0.946480,0.001920
8,23.369699,1.239097,0.038652,0.005809,0.6,16,3,16,0.05,2,0.8,"{'subsample': 0.6, 'scale_pos_weight': 16, 'mi...",0.859299,0.848428,0.856833,0.854854,0.004653,32,0.992701,0.992789,0.991646,0.992379,0.000519
9,12.795562,2.180710,0.039659,0.012041,1,25,3,6,0.05,0.5,1,"{'subsample': 1.0, 'scale_pos_weight': 25, 'mi...",0.871448,0.869628,0.847448,0.862841,0.010910,25,0.973914,0.973596,0.974148,0.973886,0.000226


In [ ]:
shap_values = explainer.shap_values( X_no_cat)
shap.summary_plot(shap_values,X_no_cat ,feature_names=feature_names, show=False)
#plt.savefig("shap_summary.svg", format='svg', dpi=300, bbox_inches='tight')
for predictor in feature_names:
    if predictor != 'age':
        save = shap.dependence_plot(predictor, shap_values,X_no_cat ,feature_names=feature_names,interaction_index='age', show=False)
    else:
        save = shap.dependence_plot(predictor, shap_values,X_no_cat ,feature_names=feature_names, interaction_index='gender', show=False)
        
    # plt.savefig(predictor+".svg", format='svg', dpi=300, bbox_inches='tight')

# LightGBM

In [7]:
LGB_classifier = lgb.LGBMClassifier()
LGB_params = {
             'num_leaves': [1,5,8,10,15,20,35,40], 
             'min_child_samples': [1,5,10,20,50,100,200,300,400,500], 
             'min_child_weight': [1e-5, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4],
             'subsample': [0.2,0,4,0.5, 0.6, 0.8, 1.0],
             'colsample_bytree': [0.6, 0.8, 1.0],
             'reg_alpha': [0, 1e-1, 1, 2, 5, 7, 10, 50, 100],
             'reg_lambda': [0, 1e-1, 1, 5, 10, 20, 50, 100],
             'scale_pos_weight':  list(range(scale_pos_weight_min,3*scale_pos_weight_min))

}
model = strat_cv_it(LGB_classifier ,LGB_params,True,50)
explainer = shap.TreeExplainer(model)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    4.4s
[Parallel(n_jobs=4)]: Done 120 tasks      | elapsed:   15.3s
[Parallel(n_jobs=4)]: Done 150 out of 150 | elapsed:   18.8s finished


0.8858988349300331

{'subsample': 0.8,
 'scale_pos_weight': 17,
 'reg_lambda': 10,
 'reg_alpha': 50,
 'num_leaves': 8,
 'min_child_weight': 100.0,
 'min_child_samples': 400,
 'colsample_bytree': 1.0}

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_subsample,param_scale_pos_weight,param_reg_lambda,param_reg_alpha,param_num_leaves,param_min_child_weight,param_min_child_samples,param_colsample_bytree,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,mean_train_score,std_train_score
0,0.008113,0.000515,0.000000,0.000000,0,21,0,10,15,0.001,10,1,"{'subsample': 0, 'scale_pos_weight': 21, 'reg_...",0.000000,0.000000,0.000000,0.000000,0.000000,33,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.074430,0.024794,0.014729,0.001993,0.2,14,0,5,10,1e-05,500,1,"{'subsample': 0.2, 'scale_pos_weight': 14, 're...",0.890794,0.872985,0.877198,0.880326,0.007599,7,0.897364,0.903348,0.902621,0.901111,0.002666
2,0.006814,0.000033,0.000000,0.000000,0.2,26,5,100,1,0.1,400,0.8,"{'subsample': 0.2, 'scale_pos_weight': 26, 're...",0.000000,0.000000,0.000000,0.000000,0.000000,33,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.405814,0.449143,0.012303,0.006314,0.8,22,0,10,20,1e-05,20,0.6,"{'subsample': 0.8, 'scale_pos_weight': 22, 're...",0.873668,0.864343,0.864080,0.867364,0.004459,23,0.956204,0.956985,0.956991,0.956726,0.000370
4,1.566775,0.119186,0.019389,0.002036,1,10,5,0,40,1e-05,1,1,"{'subsample': 1.0, 'scale_pos_weight': 10, 're...",0.859381,0.849073,0.839392,0.849282,0.008162,26,0.993747,0.994153,0.993009,0.993636,0.000474
5,0.331653,0.144189,0.015925,0.000788,0.8,12,100,0.1,10,0.01,20,0.8,"{'subsample': 0.8, 'scale_pos_weight': 12, 're...",0.888574,0.868840,0.874697,0.877370,0.008275,12,0.920166,0.928289,0.927105,0.925187,0.003583
6,0.088211,0.016800,0.008028,0.000391,0.6,19,100,0,15,0.01,500,1,"{'subsample': 0.6, 'scale_pos_weight': 19, 're...",0.890893,0.873207,0.878164,0.880755,0.007449,5,0.894810,0.901340,0.899182,0.898444,0.002716
7,0.118714,0.012258,0.009682,0.002480,0.8,25,100,1,15,1,50,0.8,"{'subsample': 0.8, 'scale_pos_weight': 25, 're...",0.886202,0.866347,0.875050,0.875866,0.008126,16,0.926026,0.929879,0.928077,0.927994,0.001574
8,0.045372,0.006663,0.007611,0.000921,0.6,13,20,0.1,5,0.1,500,1,"{'subsample': 0.6, 'scale_pos_weight': 13, 're...",0.891010,0.872518,0.878083,0.880537,0.007746,6,0.897539,0.903936,0.902386,0.901287,0.002725
9,0.239158,0.054626,0.008600,0.000852,0.2,26,5,7,35,0.001,1,0.8,"{'subsample': 0.2, 'scale_pos_weight': 26, 're...",0.865212,0.859092,0.856455,0.860253,0.003668,24,0.978632,0.978637,0.978503,0.978591,0.000062


In [ ]:
shap_values = explainer.shap_values(dataset.drop(columns=['expired']))
shap.summary_plot(shap_values[1],dataset.drop(columns=['expired']) ,feature_names=feature_names, show=False)
#plt.savefig("shap_summary.svg", format='svg', dpi=300, bbox_inches='tight')
for predictor in feature_names:
    if predictor != 'age':
        save = shap.dependence_plot(predictor, shap_values[1],dataset.drop(columns=['expired']) ,feature_names=feature_names,interaction_index='age', show=False)
    else:
        save = shap.dependence_plot(predictor, shap_values[1],dataset.drop(columns=['expired']) ,feature_names=feature_names, interaction_index='gender', show=False)
        
  #  plt.savefig(predictor+".svg", format='svg', dpi=300, bbox_inches='tight')